### Download Script for PlanetScope Orders

Documentation:
https://github.com/samapriya/porder

In [ ]:
import os
import glob
import pandas as pd
import shutil
import datetime
import warnings; warnings.simplefilter('ignore')
import datetime
import modules.order_utils as util
from pathlib import Path
import zipfile
import tqdm
import numpy as np

### Settings 

In [ ]:
# Set number of days to look into the past for the orders list
DATE_RANGE = 30
download_dir = 'R:\p_aicore\AICORE\planet_data_inference_grid\downloads'

#### Calculate timestamps for orders query 

In [ ]:
today = datetime.datetime.now().strftime('%Y-%m-%d')
start = (datetime.datetime.now() - datetime.timedelta(DATE_RANGE)).strftime('%Y-%m-%d')

#### Show list of running orders 

In [ ]:
running = !porder ostate --state running --start "{start}" --end "{today}"
util.show_orders(running)

#### Show list of successful orders 

In [ ]:
success = !porder ostate --state success --start "{start}" --end "{today}"
df = util.show_orders(success)
df

#### Download Order 

Select Order IDs as list or array - see table above

In [ ]:
ORDERS = np.arange(10)#[0,1,2,3,4]

for order_id in ORDERS:
    util.download_planet_order(df, order_id, download_dir)

### Desprecated
got to *PL_05_reorder_explorer_downloads*

#### Restructure downloaded files
* create subdirs for each scene

In [ ]:
util.restructure_downloaded_files(download_dir)

#### WOrkaround for Planet Explorer direct download 

In [ ]:
!porder quota

#### Unzip files

In [ ]:
download_dir = Path(r'R:\p_aicore\AICORE\planet_data_inference_grid\downloads')
target_dir = Path(r'R:\p_aicore\AICORE\planet_data_inference_grid\tiles')
dl_name = ''

In [ ]:
DIR = download_dir# / dl_name

In [ ]:
zipfiles = list(DIR.glob('*/*.zip'))

In [ ]:
zipfiles

In [ ]:
for zfile in tqdm.tqdm_notebook(zipfiles[:2]):
    try:
        util.unzip_PSOrthoTileOrder(zfile, target_dir=target_dir)
    except:
        continue